# MLP vs DCGAN on MNIST
This notebook compares the performance of two GAN architectures — a fully connected (MLP-based) GAN and a convolutional DCGAN — on the MNIST dataset.

We'll evaluate each model's training behavior and visualize generated samples over time.


In [ ]:
# Imports
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt

from models import GAN_NN_MNIST, DCGAN_MNIST
from utils import random_noise, dcgan_random_noise, train_gan_model, plot_generated_images


# **Set Seed & MLP-GAN Config**

In [ ]:
# Set seed for reproducibility
seed = 10
torch.manual_seed(seed)

# MLP-GAN Hyperparameters
num_epochs = 50
batch_size = 32
latent_dim = 100
z_mode = 'uniform'
image_size = (28, 28)
input_channel = 1
n_filters = 32
hidden_units = 256
num_layers = 3


## Load MNIST Dataset
We apply a standard normalization transform to scale pixel values to [-1, 1] range for use with Tanh activation.


In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=(0.5,), std=(0.5,))])

mnist_dataset = torchvision.datasets.MNIST(root='./', train=True, download=True, transform=transform)
train_dl = DataLoader(mnist_dataset, batch_size=batch_size, shuffle=True, drop_last=True)


## Train MLP-GAN
We use a fully connected generator and discriminator with 3 layers each.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_nn = GAN_NN_MNIST(
    input_size=latent_dim, gen_hidden_units=hidden_units, gen_num_layers=num_layers,
    gen_output_size=784, disc_hidden_units=hidden_units, disc_num_layers=num_layers,
    disc_output_size=1, dropout=0.1).to(device)

loss_fn = nn.BCEWithLogitsLoss()
gen_optimizer_nn = torch.optim.Adam(model_nn.gen_model.parameters(), lr=1e-3)
disc_optimizer_nn = torch.optim.Adam(model_nn.disc_model.parameters(), lr=1e-3)

fixed_z = random_noise(batch_size, latent_dim, z_mode).to(device)
training_progress_check = random_noise(16, latent_dim, z_mode)

all_g_losses, all_d_losses, epoch_samples = train_gan_model(
    model_nn, gen_optimizer_nn, disc_optimizer_nn, loss_fn,
    num_epochs, train_dl, latent_dim, training_progress_check,
    fixed_z, model_type='nn_gan'
)


# **Plot MLP-GAN Losses**

In [ ]:
g_losses_cpu = [g.cpu() if torch.is_tensor(g) else g for g in all_g_losses]
half_d_losses = [d.cpu()/2 if torch.is_tensor(d) else d/2 for d in all_d_losses]

plt.plot(g_losses_cpu, label='Generator loss')
plt.plot(half_d_losses, label='Discriminator loss')
plt.legend(fontsize=20)
plt.xlabel('Epoch', size=15)
plt.ylabel('Loss', size=15)
plt.title("MLP-GAN")
plt.grid()
plt.show()


# **Show MLP Images by Epoch**

In [ ]:
selected_epochs = [1, 2, 25, 42, 48, num_epochs]
fig = plt.figure(figsize=(10, 14))
plt.suptitle("MLP-GAN: Sample Images Created During Training", fontsize=12, color='blue', y=0.92)

for i, e in enumerate(selected_epochs):
    for j in range(5):
        ax = fig.add_subplot(6, 5, i*5 + j + 1)
        ax.set_xticks([])
        ax.set_yticks([])
        if j == 0:
            ax.text(-0.06, 0.5, f'Epoch {e}', rotation=90, size=18, color='red',
                    horizontalalignment='right', verticalalignment='center', transform=ax.transAxes)
        image = epoch_samples[e - 1][j]
        ax.imshow(image, cmap='gray_r')

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


# **MLP Final Output Grid**

In [ ]:
plot_generated_images(model_nn.gen_model, grid_dim=20, latent_dim=latent_dim, model_type='nn_gan')


## Train DCGAN
Next, we use a convolutional generator and discriminator, following the DCGAN design pattern.


DCGAN Setup & Training

In [ ]:
# DCGAN hyperparameters
latent_dim = 100
n_filters = 64
batch_size = 32

mnist_dataset = torchvision.datasets.MNIST(root='./', train=True, download=True, transform=transform)
train_dl = DataLoader(mnist_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

model_cnn = DCGAN_MNIST(latent_dim, n_filters, dropout=0.1).to(device)

loss_fn = nn.BCEWithLogitsLoss()
gen_optimizer_dc = torch.optim.Adam(model_cnn.gen_model.parameters(), lr=1e-3)
disc_optimizer_dc = torch.optim.Adam(model_cnn.disc_model.parameters(), lr=1e-3)

fixed_z = dcgan_random_noise(batch_size, latent_dim, z_mode).to(device)
training_progress_check = dcgan_random_noise(16, latent_dim, z_mode)

all_g_loss_cnn, all_d_loss_cnn, epoch_samples_cnn = train_gan_model(
    model_cnn, gen_optimizer_dc, disc_optimizer_dc, loss_fn,
    num_epochs, train_dl, latent_dim, training_progress_check,
    fixed_z, model_type='dc_gan'
)


# **DCGAN Loss Plots**

In [ ]:
g_losses_cpu_cnn = [g.cpu() if torch.is_tensor(g) else g for g in all_g_loss_cnn]
half_d_loss_cnn = [d.cpu()/2 if torch.is_tensor(d) else d/2 for d in all_d_loss_cnn]

plt.plot(g_losses_cpu_cnn, label='Generator loss')
plt.plot(half_d_loss_cnn, label='Discriminator loss')
plt.legend(fontsize=20)
plt.title("CNN-GAN")
plt.xlabel('Epoch', size=15)
plt.ylabel('Loss', size=15)
plt.grid()
plt.show()


# **DCGAN Sample Images**

In [ ]:
selected_epochs = [1, 2, 25, 42, 48, num_epochs]
fig = plt.figure(figsize=(10, 14))
plt.suptitle("CNN-GAN: Sample Images Created During Training", fontsize=12, color='blue', y=0.92)

for i, e in enumerate(selected_epochs):
    for j in range(5):
        ax = fig.add_subplot(6, 5, i*5 + j + 1)
        ax.set_xticks([])
        ax.set_yticks([])
        if j == 0:
            ax.text(-0.06, 0.5, f'Epoch {e}', rotation=90, size=18, color='red',
                    horizontalalignment='right', verticalalignment='center', transform=ax.transAxes)
        image = epoch_samples_cnn[e - 1][j]
        ax.imshow(image, cmap='gray_r')

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


# **DCGAN Final Grid**

In [ ]:
plot_generated_images(model_cnn.gen_model, grid_dim=20, latent_dim=latent_dim, model_type='cnn_gan')


##  Conclusion

Both GANs are capable of learning to generate handwritten digits from the MNIST dataset.

- **MLP-GAN** converges reasonably but often produces less sharp images.
- **DCGAN** learns spatial features better and generates more realistic digits earlier in training.

If image quality is a priority, **DCGAN** is the better architecture due to its inductive bias for images.

You've now compared two core GAN architectures side-by-side on MNIST. Feel free to tweak hyperparameters or test on new datasets!
